<a href="https://colab.research.google.com/github/kavyaaaa16/Sentiment_analysis/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets --quiet

import pandas as pd
import tensorflow as tf
from transformers import BertTokenizerFast, TFBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


In [3]:
twitter=pd.read_csv('/content/twitter_training.csv')

In [4]:
twitter.columns=['id', 'category','label', 'review']
twitter

,id,category,label,review
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [5]:
twitter.isnull().sum()

,0
id,0
category,0
label,0
review,686


In [6]:
twitter.dropna(subset=['review'],inplace=True)
twitter.isnull().sum()

,0
id,0
category,0
label,0
review,0


In [7]:
twitter.drop(['id', 'category'], axis='columns')

,label,review
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...
...,...,...
74676,Positive,Just realized that the Windows partition of my...
74677,Positive,Just realized that my Mac window partition is ...
74678,Positive,Just realized the windows partition of my Mac ...
74679,Positive,Just realized between the windows partition of...


In [8]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 10.7 MB/s eta 0:00:00


In [9]:
import re
import emoji

def cleaning(text):
  text=text.lower()
  text=emoji.replace_emoji(text, replace='')
  text=re.sub(r"http\S+", "", text)
  text= re.sub(r"[^a-zA-Z0-9\s]", "", text)
  text=re.sub(r"(.)\1{2,}", r"\1", text)
  text=re.sub(r"\s+", " ", text).strip()
  return text

twitter['review']=twitter['review'].astype(str).apply(cleaning)

In [10]:
valid_labels=['Positive', 'Negative', 'Neutral']
twitter=twitter[twitter['label'].isin(valid_labels)]
twitter=twitter.reset_index(drop=True)

In [11]:
twitter.shape

(61120, 4)

In [12]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

topword_list=stopwords.words('english') #will give a list of stopwords
stop=set(stopwords.words('english'))  #we convert it to set for faster checking
print(stop)

{'no', 'of', 'they', 'her', 'with', 'but', 'why', 'down', "it's", 'during', 'at', "you'd", 'a', 'an', 'above', "they'd", 'had', "they'll", 'on', 'be', 'through', 'further', "you're", "should've", 'did', "she'd", "he'd", 'isn', "needn't", 'ourselves', 'in', "it'd", 'most', "she'll", 'd', 'weren', 'here', 'we', 'our', "we'd", 'any', 'has', 'both', "hadn't", "weren't", 'shouldn', 'very', "it'll", 'than', 'some', 'as', "mustn't", 'after', 'are', "hasn't", "wouldn't", 'which', 'again', 'to', 'wouldn', 'itself', 'doesn', 'don', 'over', 'more', "she's", 'so', 'these', 'because', 'it', "they're", "couldn't", 'm', 'shan', 'yourselves', "i'm", 'ma', 'what', 'how', 'such', 'aren', "mightn't", 'same', 'should', 'hadn', 'your', "you'll", 'their', 'nor', "didn't", "i'd", 'about', 'him', 'this', 'whom', "shan't", "you've", 'was', "i'll", "we'll", 'its', 'once', 'himself', "i've", 'into', 'under', 'didn', 'up', 'or', 'have', 'ours', 'before', 'then', "wasn't", 'where', 'y', 'just', "aren't", 'between'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
def remove_stopwords(text):
  words=text.split()
  filtered=[w for w in words if w not in stop]
  return ' '.join(filtered)

twitter['review']=twitter['review'].apply(remove_stopwords)

In [14]:
test_df=pd.read_csv('/content/books_test.csv')

In [15]:
test_df=test_df.drop(['title','author', 'rating','rating_count', 'genre'], axis='columns')


In [16]:
test_df['review']=test_df['review'].apply(remove_stopwords)
test_df

,review,label
0,ive reading lot romance lately thanks arcs kno...,positive
1,book shows us become supercommunicators superc...,positive
2,tldr empire ai definitive chronicle ai revolut...,negative
3,note 4172023 someone reported unmarked spoiler...,negative
4,lovely lovely book poems nearly every poem end...,positive
...,...,...
194,rarely write reviews book feels like personal ...,negative
195,keep switching rating book 5 4 5 changing opin...,positive
196,absolutely loved rereading pet sematary experi...,positive
197,rating stars universe needed dead agreedyou ne...,positive


In [17]:
# Fix label typos and standardize to lowercase
def clean_label(label):
    label = label.lower()
    if label == 'neagtive':
        return 'negative'
    return label

twitter['label'] = twitter['label'].apply(clean_label)
test_df['label'] = test_df['label'].apply(clean_label)


In [18]:
def head_tail_tokenize(texts, tokenizer, head_len=96, tail_len=32, max_length=128):
    all_input_ids = []
    all_attention_mask = []

    for text in texts:
        tokens = tokenizer.tokenize(text)

        if len(tokens) > max_length:
            head_tokens = tokens[:head_len]
            tail_tokens = tokens[-tail_len:]
            combined_tokens = head_tokens + tail_tokens
        else:
            combined_tokens = tokens

        # Convert tokens to ids
        input_ids = tokenizer.convert_tokens_to_ids(combined_tokens)

        # Add special tokens ([CLS] and [SEP]) if tokenizer uses them
        # For BERT-style models, usually:
        input_ids = [tokenizer.cls_token_id] + input_ids + [tokenizer.sep_token_id]

        # Pad or truncate to max_length
        if len(input_ids) > max_length:
            input_ids = input_ids[:max_length]
        else:
            input_ids += [tokenizer.pad_token_id] * (max_length - len(input_ids))

        # Attention mask: 1 for real tokens, 0 for padding
        attention_mask = [1 if id != tokenizer.pad_token_id else 0 for id in input_ids]

        all_input_ids.append(input_ids)
        all_attention_mask.append(attention_mask)

    return {'input_ids': all_input_ids, 'attention_mask': all_attention_mask}


In [19]:
train_texts = twitter['review'].astype(str).tolist()
train_labels = twitter['label'].tolist()

test_texts = test_df['review'].astype(str).tolist()
test_labels = test_df['label'].tolist()


In [20]:

# Convert both label lists to string
train_labels = [str(label) for label in train_labels]
test_labels = [str(label) for label in test_labels]

# Combine labels before fitting
all_labels = train_labels + test_labels

label_encoder = LabelEncoder()
label_encoder.fit(all_labels)

# Then transform
y_train = to_categorical(label_encoder.transform(train_labels))
y_test = to_categorical(label_encoder.transform(test_labels))



In [21]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = head_tail_tokenize(train_texts, tokenizer, head_len=96, tail_len=32, max_length=64)
test_encodings = head_tail_tokenize(test_texts, tokenizer, head_len=96, tail_len=32, max_length=64)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [22]:
import tensorflow as tf
batch_size = 16
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).shuffle(len(train_texts)).batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)





In [23]:
from transformers import TFDistilBertForSequenceClassification
model = TFDistilBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['bert.encoder.layer.4.intermediate.dense.bias', 'bert.encoder.layer.3.output.dense.weight', 'bert.encoder.layer.5.intermediate.dense.weight', 'bert.encoder.layer.8.output.LayerNorm.weight', 'bert.encoder.layer.6.output.dense.bias', 'bert.encoder.layer.1.attention.output.dense.bias', 'bert.encoder.layer.4.output.dense.bias', 'bert.encoder.layer.4.attention.self.query.weight', 'bert.encoder.layer.2.attention.self.query.weight', 'bert.encoder.layer.8.output.dense.bias', 'bert.encoder.layer.1.output.LayerNorm.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.5.output.LayerNorm.weight', 'bert.encoder.layer.1.attention.output.LayerNorm.bias', 'bert.encoder.layer.4.output.LayerNorm.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.bias', 'bert.encoder.layer.6.attention.self.key.weight', 'bert.encoder.layer.9.output.LayerNorm.bias', 'bert.encode

In [24]:
import tensorflow as tf

loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']


In [25]:
from transformers import TFBertForSequenceClassification
from transformers import AdamWeightDecay
from tensorflow.keras.losses import CategoricalCrossentropy

# Load model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Create optimizer with Hugging Face-compatible AdamW
optimizer = AdamWeightDecay(learning_rate=3e-5, weight_decay_rate=0.01)

# Set loss and metrics
loss = CategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']

# Compile model (this will now work)
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
epochs = 3

model.fit(train_dataset, validation_data=test_dataset, epochs=1)


KeyboardInterrupt: 

In [27]:
# Create a directory to save model
model_save_path = "./bert-sentiment-model"
tokenizer_save_path = "./bert-sentiment-tokenizer"

# Save model
model.save_pretrained(model_save_path)

# Save tokenizer
tokenizer.save_pretrained(tokenizer_save_path)


('./bert-sentiment-tokenizer/tokenizer_config.json',
 './bert-sentiment-tokenizer/special_tokens_map.json',
 './bert-sentiment-tokenizer/vocab.txt',
 './bert-sentiment-tokenizer/added_tokens.json')

In [79]:
model.save_pretrained('/content/sentiment_bert_model_headtail')
tokenizer.save_pretrained('/content/sentiment_bert_model_headtail')


('/content/sentiment_bert_model_headtail/tokenizer_config.json',
 '/content/sentiment_bert_model_headtail/special_tokens_map.json',
 '/content/sentiment_bert_model_headtail/vocab.txt',
 '/content/sentiment_bert_model_headtail/added_tokens.json')

In [49]:
model.save_pretrained("model_checkpoint_v1")
tokenizer.save_pretrained("model_checkpoint_v1")


('model_checkpoint_v1/tokenizer_config.json',
 'model_checkpoint_v1/special_tokens_map.json',
 'model_checkpoint_v1/vocab.txt',
 'model_checkpoint_v1/added_tokens.json')

In [32]:
from transformers import TFBertForSequenceClassification, BertTokenizer

model = TFBertForSequenceClassification.from_pretrained(
    "/content/sentiment_bert_model_headtail",
    local_files_only=True  # 🔐 forces local loading, no HF hub call
)

tokenizer = BertTokenizer.from_pretrained(
    "/content/sentiment_bert_model_headtail",
    local_files_only=True
)


OSError: Can't load the configuration of '/content/sentiment_bert_model_headtail'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/content/sentiment_bert_model_headtail' is the correct path to a directory containing a config.json file

In [34]:
model.evaluate(test_dataset)


13/13 [==============================] - 41s 3s/step - loss: 1.6754 - accuracy: 0.3719


[1.6754401922225952, 0.37185928225517273]

In [35]:
weak_df=pd.read_csv('/content/train_labeled.csv')
weak_df=weak_df.drop(['title','author', 'rating','rating_count', 'genre'], axis='columns')
weak_df['label'] = weak_df['label'].str.strip().str.capitalize()
# --- Step 1: Prepare your weak dataset ---
# Convert reviews to strings (if needed)
weak_df['review'] = weak_df['review'].astype(str)
weak_df = weak_df[weak_df['label'].isin(['Positive', 'Negative', 'Neutral'])]
weak_df = weak_df.dropna(subset=['review'])
weak_df = weak_df[weak_df['review'].str.strip() != '']
weak_df = weak_df.reset_index(drop=True)

# Apply preprocessing
weak_df['review'] = weak_df['review'].apply(remove_stopwords)
weak_df['label'] = weak_df['label'].apply(clean_label)




# Tokenize
weak_encodings = tokenizer(
    weak_df['review'].tolist(),
    truncation=True,
    padding=True,
    max_length=128,
    return_tensors='tf'
)

# --- Step 2: Encode weak labels (if available) ---
if 'label' in weak_df.columns:
    weak_labels = [str(lab) for lab in weak_df['label']]
    encoded_labels = label_encoder.transform(weak_labels)
    y_weak = to_categorical(encoded_labels)
    weak_dataset = tf.data.Dataset.from_tensor_slices((dict(weak_encodings), y_weak))
else:
    # For prediction-only mode
    weak_dataset = tf.data.Dataset.from_tensor_slices(dict(weak_encodings))

# Batch it
weak_dataset = weak_dataset.batch(16).cache().prefetch(tf.data.AUTOTUNE)

# --- Step 3: Evaluate or Predict ---
if 'label' in weak_df.columns:
    # Evaluate if labels are available
    result = model.evaluate(weak_dataset)
    print(f"Weak Label Evaluation - Loss: {result[0]}, Accuracy: {result[1]}")
else:but
    # Just predict if labels not available
    predictions = model.predict(weak_dataset).logits
    predicted_labels = np.argmax(predictions, axis=1)
    decoded_labels = label_encoder.inverse_transform(predicted_labels)
    weak_df['predicted_label'] = decoded_labels
    print(weak_df[['review', 'predicted_label']].head())


 7/71 [=>............................] - ETA: 8:15 - loss: 1.6250 - accuracy: 0.3125

KeyboardInterrupt: 